In [1]:
import numpy as np
import pandas as pd
#import fiona 
#import math
import xarray as xa
#import matplotlib.pyplot as plt
#%matplotlib inline
#from matplotlib import cm
#from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import datetime
import random
#from scipy.interpolate import interp1d
import geopandas as gpd

# Change the path from paramdb_markstro to paramdb_master when the jh_coef_hru values are actually computed from the real temperature data

In [16]:
#radadj_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_radadj_GF_v1.1.csv'
#radadj_solf_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_radadj_solf_GF_v1.1.csv'
#pptadj_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_pptadj_GF_v1.1.csv'
gdb_path = 'c:/Users/markstro/work1.1/GIS/GFv1.1_v2e.gdb'

markstro_param_path = "c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_markstro/"
path = 'c:/Users/markstro/work1.1/weather/'
farns_v11_path = 'c:/Users/markstro/work1.1/GIS/farns_gf_1_1_mm_per_month.shp'
jh_coef_hru_fn = 'c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_markstro/jh_coef_hru.csv'
nrel_shapefile_fn = "c:/Users/markstro/work1.1/GIS/nhm_shapefiles/nhm_hru_gf1_1_nrel_solrad.shp"

In [3]:
#fns = [("2011", "2011_gm_climate_2020_03_02.nc"), ("2012", "2012_gm_climate_2020_03_03.nc"),
#       ("2013", "2013_gm_climate_2020_03_05.nc"), ("2014", "2014_gm_climate_2020_03_06.nc"),
#       ("2015", "2015_gm_climate_2020_03_08.nc"), ("2016", "2016_gm_climate_2020_03_09.nc"),
#       ("2017", "2017_gm_climate_2020_03_10.nc"), ("2018", "2018_gm_climate_2020_03_12.nc")]

fns = [("2015", "2015_gm_climate_2020_03_08.nc")]

In [4]:
# Read the v11 Farnsworth shapefile
# The PET values (cols jan - dec) are in units of mm / month
farns_v11 = gpd.read_file(farns_v11_path)

In [5]:
farns_v11.head()

,nhru_v11,nhm_id_v10,dist,Unnamed_ 0,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,geometry
0,76127,76128,1.148310,76127,49.945515,60.029290,91.94805,122.314926,158.02434,172.70332,181.95477,167.50844,134.82234,104.785355,70.874160,53.167057,"POLYGON ((-105544.567 804074.976, -105542.972 ..."
1,76147,76148,4.135990,76147,50.172657,60.212677,92.15552,122.602650,158.38263,173.02405,182.35974,167.88830,135.17688,105.129745,71.116910,53.408350,"POLYGON ((-97185.217 806355.005, -97184.805 80..."
2,76170,76171,3.466319,76170,48.300198,58.800156,91.40010,122.800026,159.49995,174.69990,184.19990,169.39993,135.90000,104.900085,70.000145,51.700195,"MULTIPOLYGON (((-106590.453 815804.909, -10646..."
3,76172,76173,2.518344,76172,50.242560,60.354050,92.60870,123.404350,159.62950,174.57050,184.11366,169.57266,136.54752,106.108700,71.683556,53.642563,"POLYGON ((-106762.641 815925.020, -106754.831 ..."
4,76181,76182,3.864358,76181,51.480717,61.344570,93.39392,123.848880,159.81587,174.62144,184.20695,169.84592,137.07890,106.982970,72.801120,54.910404,"POLYGON ((-109785.311 816675.047, -109784.942 ..."


In [6]:
farns_v11.tail()

,nhru_v11,nhm_id_v10,dist,Unnamed_ 0,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,geometry
114953,57964,57965,72.050651,57964,4.287964,17.662860,49.587770,85.91264,125.675050,145.599980,156.34999,141.46255,106.737670,71.025300,33.150390,9.825459,"MULTIPOLYGON (((-294975.252 2728035.068, -2950..."
114954,64080,64081,22.302546,64080,0.000000,6.210107,33.820175,67.23024,103.830220,123.575180,134.55504,122.38992,91.679855,58.969795,23.969807,1.179889,"POLYGON ((-1284185.000 2658495.000, -1284185.0..."
114955,64150,64151,1.124793,64150,0.000000,9.088320,36.233490,68.61729,104.197920,123.148346,133.77536,121.82024,91.866630,60.176285,26.038994,4.148405,"POLYGON ((-1347645.097 2651834.819, -1347614.8..."
114956,65633,65634,2.190821,65633,9.794936,20.508738,50.327328,85.22777,124.662056,145.593460,158.36440,146.10867,113.456410,79.473755,41.462563,17.411674,"POLYGON ((-1017435.000 2869395.054, -1017435.0..."
114957,18843,18842,2518.518508,18841,17.100000,25.400000,45.900000,66.50000,89.200000,98.700000,103.50000,93.30000,71.800000,51.400000,30.000000,18.400000,"POLYGON ((437685.115 2806784.914, 437685.231 2..."


In [26]:
days_in_month = [31,28,31,30,31,30,31,31,30,31,30,31]

[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


In [28]:
# Extract the PET values and convert from mm/month to inches/month

farns_vals = farns_v11[['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']].values

for imon in range(nmonth):
    farns_vals[:,imon] = farns_vals[:,imon] / 25.4 / days_in_month[imon]
    
print(farns_vals)
print(farns_vals.shape)
nhru = farns_vals.shape[0]
nmonth = farns_vals.shape[1]
print(nhru, nmonth)

[[0.06343093 0.08440564 0.11677426 ... 0.13307767 0.09301071 0.0675223 ]
 [0.0637194  0.08466349 0.11703774 ... 0.13351504 0.09332928 0.06782874]
 [0.06134137 0.08267738 0.11607836 ... 0.13322337 0.09186371 0.06565938]
 ...
 [0.         0.01277885 0.04601662 ... 0.07642403 0.03417191 0.00526848]
 [0.01243959 0.02883681 0.06391583 ... 0.10093187 0.05441281 0.02211287]
 [0.02171704 0.03571429 0.05829312 ... 0.06527813 0.03937008 0.02336805]]
(114958, 12)
114958 12


In [29]:
# Extract the Farnsworth v11 HRU order
farns_nhru_v11 = farns_v11['nhru_v11'].values
print(farns_nhru_v11)

[76127 76147 76170 ... 64150 65633 18843]


In [30]:
# read the jh_coef_hru parameters

# read the jh_coef_hru values from the csv file produced by jh_coef_hru.ipynb
jh_coef_hru = pd.read_csv(jh_coef_hru_fn)
jh_coef_hru.head()

,$id,jh_coef_hru
0,1,8.666409
1,2,17.340042
2,3,1.827372
3,4,21.176576
4,5,8.729901


In [31]:
jh_coef_hru_vals = jh_coef_hru['jh_coef_hru']
jh_coef_hru_idx = jh_coef_hru['$id']

# Need to read the ncf files for actual tmin values

In [32]:
#prcp = xa.Dataset()

#ds = xa.open_dataset(path + "2015_gm_climate_2020_03_08.nc")
#tmin = ds["tmin"]

#dates = [datetime.datetime(2015, 1, 1), datetime.datetime(2015, 1, 2), datetime.datetime(2015, 1, 3),
#         datetime.datetime(2015, 1, 4)]
#ndays = len(dates)

date1 = '2015-01-01'
date2 = '2015-12-31'
dates = pd.date_range(date1, date2).tolist()
ndays = len(dates)

foo = np.random.random_sample(((nhru*ndays),))
foo = foo * 40.0
foo.shape = (ndays, nhru)

tminf = foo
print(tminf)

[[23.00508066 26.01372607 18.74471481 ... 37.82899518 21.75229858
  35.96726127]
 [15.58833421  0.26833266 14.01933189 ... 10.99698645 11.39452803
  35.24175535]
 [ 1.08278957 29.2045061  25.04572407 ...  5.22119758  5.17082498
  34.47123765]
 ...
 [20.31691027 17.85156596  2.64800504 ... 16.14776067 39.53411974
  18.87186614]
 [33.16463727 29.91575093 20.61875039 ... 16.98580368  8.40948267
  15.88833358]
 [14.60697016 36.77531173 39.22303051 ... 33.1903624  18.95929547
  21.02946196]]


In [33]:
foo = np.random.random_sample(((nhru*ndays),))
foo = foo * 10.0
foo.shape = (ndays, nhru)

tmaxf = tminf + foo
print(tmaxf)
print(tmaxf.shape)

[[25.93217233 30.44955227 26.8972261  ... 44.49229132 31.46762563
  43.51058494]
 [22.25263001  3.40727486 22.91158856 ... 13.82028425 18.99597326
  36.75155772]
 [ 5.89512019 32.51677607 30.36152446 ... 10.6130623   6.82807524
  36.51508649]
 ...
 [27.88409094 21.10615456  6.02293091 ... 18.03256702 40.85789832
  18.90754715]
 [41.25464489 35.78552006 26.54540078 ... 26.634075   10.40594503
  18.1380528 ]
 [15.72706511 44.25252812 39.94736553 ... 36.31824879 19.20785624
  29.9248956 ]]
(365, 114958)


In [34]:
tavgf = (tmaxf + tminf) / 2.0
print(tavgf)
print(tavgf.shape)

[[24.4686265  28.23163917 22.82097045 ... 41.16064325 26.6099621
  39.73892311]
 [18.92048211  1.83780376 18.46546022 ... 12.40863535 15.19525064
  35.99665653]
 [ 3.48895488 30.86064108 27.70362427 ...  7.91712994  5.99945011
  35.49316207]
 ...
 [24.10050061 19.47886026  4.33546797 ... 17.09016384 40.19600903
  18.88970664]
 [37.20964108 32.85063549 23.58207559 ... 21.80993934  9.40771385
  17.01319319]
 [15.16701763 40.51391992 39.58519802 ... 34.75430559 19.08357586
  25.47717878]]
(365, 114958)


In [35]:
#(32°F − 32) × 5/9 = 0°C

tavgc = (tavgf - 32.0) * 5.0 / 9.0
print(tavgc)
print(tavgc.shape)

[[ -4.18409639  -2.0935338   -5.09946086 ...   5.08924625  -2.9944655
    4.29940173]
 [ -7.26639883 -16.75677569  -7.51918876 ... -10.88409147  -9.33597187
    2.22036474]
 [-15.83946951  -0.63297718  -2.38687541 ... -13.37937226 -14.44474994
    1.9406456 ]
 ...
 [ -4.38861077  -6.95618875 -15.36918446 ...  -8.28324231   4.55333835
   -7.28349631]
 [  2.89424504   0.47257527  -4.67662467 ...  -5.66114481 -12.55127008
   -8.32600379]
 [ -9.35165687   4.72995551   4.2139989  ...   1.53016977  -7.17579119
   -3.62378956]]
(365, 114958)


In [36]:

# Read in the monthly short wave data from NREL. It's in the shapefile that was created by running zonal means
# on the monthly DNI geotifs downloaded from NREL.

nrel_solrad = gpd.read_file(nrel_shapefile_fn)

In [37]:
# Extract the NREL monthly solrad targets from the dataframe into a numpy 2D array for the radadj calculation below.

solrad_targets_vals= nrel_solrad[["dni_jan_me", "dni_feb_me", "dni_mar_me", "dni_apr_me", "dni_may_me", "dni_jun_me",
                    "dni_jul_me", "dni_aug_me", "dni_sep_me", "dni_oct_me", "dni_nov_me", "dni_dec_me"]].values
print(solrad_targets_vals)

[[182.20148911 183.51872029 191.96332414 ... 229.74948359 196.08214547
  166.2665358 ]
 [182.0873538  182.82732109 190.77398132 ... 227.66198595 193.96723928
  164.80365733]
 [183.55624823 184.54553207 191.84952426 ... 228.18403367 194.83623904
  165.23211128]
 ...
 [100.20052231 130.79780146 165.89017402 ... 176.22581824 124.65252498
   94.91159392]
 [106.98863186 148.58928652 178.98350311 ... 177.62528297 130.36114766
  109.73921902]
 [128.07736674 176.15929126 217.490094   ... 137.29780348  98.5662098
  104.56964488]]


In [38]:
# The NREL solrad values are in units of kWh/m2/Day. PRMS uses Langleys per day. The conversion factor comes from
# https://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/GEM/SolarRadConversion.pdf

#                    W-sec    1 KW     1 hour               KW-hours
# 1 Langley = 41868 -------  ------   -------  =   0.6978  ---------
#                     m2     1000 W   60 sec                  m2

solrad_targets_vals_langleys = solrad_targets_vals / 0.6978
print(solrad_targets_vals_langleys)
print(solrad_targets_vals_langleys.shape)

[[261.10846819 262.99615978 275.09791364 ... 329.24832844 281.00049508
  238.27247893]
 [260.9449037  262.0053326  273.39349574 ... 326.25678697 277.96967509
  236.17606381]
 [263.04994014 264.46765845 274.93482984 ... 327.00492071 279.21501725
  236.79007062]
 ...
 [143.59490156 187.443109   237.73312414 ... 252.5448814  178.63646457
  136.0154685 ]
 [153.32277424 212.93964822 256.49685169 ... 254.55041984 186.81735119
  157.26457297]
 [183.54452098 252.44954322 311.67969905 ... 196.75810186 141.25280853
  149.85618355]]
(114958, 12)


# In the line

# jh_coef_by_day[ihru, iday] = farns_vals[ihru, imon] / (tavgf[iday,ihru] - jh_coef_hru_vals[ihru])

# anywhere the ihru index is used, it must still be mapped!!

In [39]:
# DO j = 1, Active_hrus
#          elh = (597.3-(0.5653*Tavgc(i)))*2.54
#          Potet(i) = Jh_coef(i, Nowmonth)*(Tavgf(i)-Jh_coef_hru(i))*Swrad(i)/elh
#          IF ( Potet(i)<0.0) Potet(i) = 0.0
#
# so
#          elh = (597.3-(0.5653*Tavgc(iday,ihru)))*2.54
#          Jh_coef(ihru, imon) = farns(ihru, imon) / ((Tavgf(iday,ihru)-Jh_coef_hru(ihru)) * Swrad(imon,ihru) / elh)

jh_coef_by_day = np.zeros(nhru * ndays)
jh_coef_by_day.shape = (nhru, ndays)

jh_coef = np.zeros(nhru * nmonth)
jh_coef.shape = (nhru, nmonth)

month_count = np.zeros(nmonth)

iday = 0
for date in dates:
    imon = date.month - 1
    jday = date.day - 1
    print(date, imon, jday, iday)
    for ihru in range(nhru):
        elh = (597.3 - (0.5653 * tavgc[iday, ihru])) * 2.54
        jh_coef_by_day[ihru, iday] = farns_vals[ihru, imon] / ((tavgf[iday,ihru] - jh_coef_hru_vals[ihru]) \
                                                               * solrad_targets_vals_langleys[ihru,imon] / elh)
        
    iday += 1

iday = 0
for date in dates:
    imon = date.month - 1
    jday = date.day - 1
    for ihru in range(nhru):
        jh_coef[ihru, imon] += jh_coef_by_day[ihru, iday]
        month_count[imon] += 1
    iday += 1

for imon in range(nmonth):
    jh_coef[:,imon] = jh_coef[:,imon] / month_count[imon]

print(jh_coef)
        


2015-01-01 00:00:00 0 0 0
2015-01-02 00:00:00 0 1 1
2015-01-03 00:00:00 0 2 2
2015-01-04 00:00:00 0 3 3
2015-01-05 00:00:00 0 4 4
2015-01-06 00:00:00 0 5 5
2015-01-07 00:00:00 0 6 6
2015-01-08 00:00:00 0 7 7
2015-01-09 00:00:00 0 8 8
2015-01-10 00:00:00 0 9 9
2015-01-11 00:00:00 0 10 10
2015-01-12 00:00:00 0 11 11
2015-01-13 00:00:00 0 12 12
2015-01-14 00:00:00 0 13 13
2015-01-15 00:00:00 0 14 14
2015-01-16 00:00:00 0 15 15
2015-01-17 00:00:00 0 16 16
2015-01-18 00:00:00 0 17 17
2015-01-19 00:00:00 0 18 18
2015-01-20 00:00:00 0 19 19
2015-01-21 00:00:00 0 20 20
2015-01-22 00:00:00 0 21 21
2015-01-23 00:00:00 0 22 22
2015-01-24 00:00:00 0 23 23
2015-01-25 00:00:00 0 24 24
2015-01-26 00:00:00 0 25 25
2015-01-27 00:00:00 0 26 26
2015-01-28 00:00:00 0 27 27
2015-01-29 00:00:00 0 28 28
2015-01-30 00:00:00 0 29 29
2015-01-31 00:00:00 0 30 30
2015-02-01 00:00:00 1 0 31
2015-02-02 00:00:00 1 1 32
2015-02-03 00:00:00 1 2 33
2015-02-04 00:00:00 1 3 34
2015-02-05 00:00:00 1 4 35
2015-02-06 00:00:

2015-10-18 00:00:00 9 17 290
2015-10-19 00:00:00 9 18 291
2015-10-20 00:00:00 9 19 292
2015-10-21 00:00:00 9 20 293
2015-10-22 00:00:00 9 21 294
2015-10-23 00:00:00 9 22 295
2015-10-24 00:00:00 9 23 296
2015-10-25 00:00:00 9 24 297
2015-10-26 00:00:00 9 25 298
2015-10-27 00:00:00 9 26 299
2015-10-28 00:00:00 9 27 300
2015-10-29 00:00:00 9 28 301
2015-10-30 00:00:00 9 29 302
2015-10-31 00:00:00 9 30 303
2015-11-01 00:00:00 10 0 304
2015-11-02 00:00:00 10 1 305
2015-11-03 00:00:00 10 2 306
2015-11-04 00:00:00 10 3 307
2015-11-05 00:00:00 10 4 308
2015-11-06 00:00:00 10 5 309
2015-11-07 00:00:00 10 6 310
2015-11-08 00:00:00 10 7 311
2015-11-09 00:00:00 10 8 312
2015-11-10 00:00:00 10 9 313
2015-11-11 00:00:00 10 10 314
2015-11-12 00:00:00 10 11 315
2015-11-13 00:00:00 10 12 316
2015-11-14 00:00:00 10 13 317
2015-11-15 00:00:00 10 14 318
2015-11-16 00:00:00 10 15 319
2015-11-17 00:00:00 10 16 320
2015-11-18 00:00:00 10 17 321
2015-11-19 00:00:00 10 18 322
2015-11-20 00:00:00 10 19 323
2015

In [40]:
f = open(markstro_param_path + 'jh_coef_hru.csv', "w")

In [41]:
f.write("$id,jh_coef_hru\n")

16

In [42]:
# The indexes being written are in the order of the values in the tmin ncf files. Once those are available, these will
# need to be reordered.

for jj in range(nhru):
    f.write(str(jj+1) + "," + str(jh_coef_hru[jj]) + '\n')

KeyError: 0

In [ ]:
f.close()